In [2]:
%cd /home/dyllanjr/Documents/Classification-M2/notebooks
from pyfuncs import processing
import numpy as np
import pandas as pd

/home/dyllanjr/Documents/Classification-M2/notebooks


In [2]:
def select_random_rows(n_rows, X, y):
    '''Returns n_rows of randomly sampled X, y and the row index'''
    row_ixs = np.random.randint(0, X.shape[0], n_rows)
    return X.iloc[row_ixs], y.iloc[row_ixs], row_ixs

In [3]:
def sample_from_all_house(n_rows_per_df):
    csvs = np.arange(101, 131)
    #unsampled csh's are 101,120,123,113
    del_csvs = np.array([124, 127, 130, 101,120,123,113])
    csvs = np.setdiff1d(csvs, del_csvs)
    sample_ix_dict = {}
    X_samps, y_samps = pd.DataFrame(), pd.Series()
    for csv in csvs:
        X, y =  processing.load_house(csv)
        X_samp, y_samp, row_ix_samp = select_random_rows(n_rows_per_df, X, y)
        del X
        del y
        X_samps = pd.concat([X_samps, X_samp])
        y_samps = pd.concat([y_samps, y_samp])
        del X_samp
        del y_samp
        sample_ix_dict[csv] = row_ix_samp
        del row_ix_samp
    return X_samps, y_samps, sample_ix_dict

In [4]:
testX, testy, ix_dict = sample_from_all_house(100000)

<ipython-input-3-415b387d6d00>:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  X_samps, y_samps = pd.DataFrame(), pd.Series()


In [37]:
columns = ['lastSensorEventHours', 'lastSensorEventSeconds',
       'lastSensorDayOfWeek', 'windowDuration', 'timeSinceLastSensorEvent',
       'prevDominantSensor1', 'prevDominantSensor2', 'lastSensorID',
       'lastSensorLocation', 'lastMotionLocation', 'complexity',
       'activityChange', 'areaTransitions', 'numDistinctSensors',
       'sensorCount-Bathroom', 'sensorCount-Bedroom', 'sensorCount-Chair',
       'sensorCount-DiningRoom', 'sensorCount-Hall', 'sensorCount-Ignore',
       'sensorCount-Kitchen', 'sensorCount-LivingRoom', 'sensorCount-Office',
       'sensorCount-OutsideDoor', 'sensorCount-WorkArea',
       'sensorElTime-Bathroom', 'sensorElTime-Bedroom', 'sensorElTime-Chair',
       'sensorElTime-DiningRoom', 'sensorElTime-Hall', 'sensorElTime-Ignore',
       'sensorElTime-Kitchen', 'sensorElTime-LivingRoom',
       'sensorElTime-Office', 'sensorElTime-OutsideDoor',
       'sensorElTime-WorkArea']

In [3]:
import pickle


In [39]:
data_pickle = [testX, testy, ix_dict]
%pwd
pickle.dump(data_pickle, open('sampled_house_data.pkl', 'wb'))

The problem with randomly sampling with the house datasets is that it allows a form of data leakage because the activity in the row following a previously trained row is likely to be the activity of the previously trained row. Thus having the row directly following a trained row in the test set would be bad practice. Instead I'll portion off the first N rows of each housing set H and then delete M rows after N leaving H - M - N rows left for each house set. Furthermore, I'm interested in generalizing the model to houses it hasn't seen yet, so I'll use houses [101,120,123,113] as completely untouched "test houses"

In [2]:
testX, testy = processing.load_house(110)

/home/dyllanjr/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
testX.shape

(273446, 36)

In [4]:
def select_first_n_rows(n_rows, X, y):
    return X.iloc[0:n_rows], y.iloc[0:n_rows]

In [9]:
def no_data_leakage_sample_from_all_houses(n_rows_per_df):
    csvs = np.arange(101, 111)
    #unsampled csh's are 101, 105, 110
    del_csvs = np.array([101, 105, 110])
    csvs = np.setdiff1d(csvs, del_csvs)
    X_samps, y_samps = pd.DataFrame(), pd.Series()
    for csv in csvs:
        X, y =  processing.load_house(csv)
        X_samp, y_samp = select_first_n_rows(n_rows_per_df, X, y)
        del X
        del y
        X_samps = pd.concat([X_samps, X_samp])
        y_samps = pd.concat([y_samps, y_samp])
        del X_samp
        del y_samp
    return X_samps, y_samps

In [10]:
trainX, trainy = no_data_leakage_sample_from_all_houses(n_rows_per_df=100000)
%cd /home/dyllanjr/Documents/Classification-M2/data/processed/
data_pickle = [trainX, trainy]
import pickle
pickle.dump(data_pickle, open('sampled_house_data_first_100k.pkl', 'wb'))

<ipython-input-9-8d22b6598c1e>:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  X_samps, y_samps = pd.DataFrame(), pd.Series()


/home/dyllanjr/Documents/Classification-M2/notebooks


In [8]:
%cd /home/dyllanjr/Documents/Classification-M2/data/processed/
data = pickle.load(open('sampled_house_data_first_100k.pkl', 'rb'))
X = data[0]
y = data[1]

/home/dyllanjr/Documents/Classification-M2/data/processed


In [9]:
y.value_counts()
#I would like to try to oversample to fix the class imbalance with SMOTE

Other_Activity      288299
Cook                111052
Entertainment        60282
Personal_Hygeine     48246
Sleep                39150
Eat                  38235
Clean                36679
Work                 31767
Toilet               21140
Dress                16520
Meds                  4683
Enter_Home            3947
dtype: int64

In [10]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_res, y_res = sm.fit_resample(X, y)

In [12]:
data_pickle = [X_res, y_res]

In [13]:
%cd /home/dyllanjr/Documents/Classification-M2/data/processed/
pickle.dump(data_pickle, open('oversampled_house_data_first_100k.pkl', 'wb'))

/home/dyllanjr/Documents/Classification-M2/data/processed
